Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [9]:
# df = pd.DataFrame()
lr =1e-5
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 1500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([0.07,#0.01,
                            ]):
        for dropout in [ 0.9
                       ]:
            for noise in [0.5#None, 100,10, 1, 
                      ]:
                for layers in [
                    [200, 60, 40]
                              ]:
                    for run in range(2):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=30,
                                             lr=lr,
                                             noise = noise,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["noise"] = noise
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
                        print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                              dresults.get('COMBINED_kmeans_ari', ""),
                              dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                              dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_noise.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3733386683665321 0.17547831242110537
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3927191844802994 0.19557531316963656

>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.43840336032404603 0.1920853841223912
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.4337800546438553 0.1349003310549292

>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3834273256851357 0.26857456509113997
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.38013749633203253 0.24514046890430685

>>>>> Data mouse_ES_cell
SCZI  0.800376096099337
(2717, 24175) (2717, 24175) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3964622674651312 0.30094329381546053
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.7399979641130798 0.27617450354604284

>>>>> Data Adam
SCZI  0.8634561030635544
(3660, 23797) (3660, 23797) keeping 1500 genes


...

.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3381384388111813 0.18114072229000497
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3400006490315689 0.17863919282768748

>>>>> Data Quake_Smart-seq2_Limb_Muscle
SCZI  0.9736759009464299
(1090, 23341) (1090, 23341) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.4666807624804577 0.20425864829256615
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.5057790938671209 0.2447794473558372

>>>>> Data Muraro
SCZI  0.7246271034758398
(2122, 19046) (2122, 19046) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.34039238746545913 0.3096167611835686
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.3371618746262583 0.3724735948757382

>>>>> Data 10X_PBMC
SCZI  0.5845649506030623
(4271, 16653) (4271, 16653) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.6023327957929326 0.3245569509567099
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.5817326873324631 0.3495034494585895

>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 1500 genes


.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.9207997641429191 0.1477805126275091
.# 0.07, 0.9, 1e-05, [200, 60, 40]   0.9347112534536195 0.11190415314054325



In [21]:
df

,dataset,dropout,kmeans_ari_0,kmeans_nmi_0,layers,leiden_ari_0,leiden_nmi_0,noise,run,temperature,time
0,Quake_Smart-seq2_Trachea,0.9,0.502064,0.580774,"[200, 60, 40]",0.162580,0.524239,0.0,0.0,0.07,3.333200
1,Quake_Smart-seq2_Trachea,0.9,0.469278,0.542781,"[200, 60, 40]",0.149636,0.521240,0.0,1.0,0.07,0.948018
2,Quake_Smart-seq2_Trachea,0.9,0.544113,0.652487,"[200, 60, 40]",0.168990,0.536053,100.0,0.0,0.07,7.421674
3,Quake_Smart-seq2_Trachea,0.9,0.525129,0.620246,"[200, 60, 40]",0.167856,0.524643,100.0,1.0,0.07,7.789366
4,Quake_Smart-seq2_Trachea,0.9,0.547560,0.655237,"[200, 60, 40]",0.173692,0.535682,10.0,0.0,0.07,7.665721
...,...,...,...,...,...,...,...,...,...,...,...
145,Muraro,0.9,0.337162,0.583480,"[200, 60, 40]",0.372474,0.616124,0.5,1.0,0.07,13.110404
146,10X_PBMC,0.9,0.602333,0.687042,"[200, 60, 40]",0.324557,0.581020,0.5,0.0,0.07,26.184781
147,10X_PBMC,0.9,0.581733,0.664460,"[200, 60, 40]",0.349503,0.575826,0.5,1.0,0.07,26.130707
148,Quake_10x_Spleen,0.9,0.920800,0.829668,"[200, 60, 40]",0.147781,0.443619,0.5,0.0,0.07,59.152132


In [20]:
df = df.fillna(0)

In [22]:
files = [
    'Quake_Smart-seq2_Trachea',
    'Quake_Smart-seq2_Diaphragm',
    'Young',
#     '10X_PBMC_select_2100',
    'mouse_ES_cell',
    'Adam',
    'Quake_10x_Bladder',
    'Quake_Smart-seq2_Lung',
#     'worm_neuron_cell_select_2100',
    'Quake_10x_Limb_Muscle',
    'worm_neuron_cell',
    'mouse_bladder_cell',
#     'mouse_ES_cell_select_2100',
    'Romanov',
    'Quake_Smart-seq2_Limb_Muscle',
    'Muraro',
#     'mouse_bladder_cell_select_2100',
    '10X_PBMC',
    'Quake_10x_Spleen',
]

In [23]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

ARI       0.745684
NMI       0.777119
sil       0.070277
run       1.000000
time    195.854597
dtype: float64

In [25]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_noise.pkl")
df = df.fillna(0)
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "noise"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).T

temperature              0.07
dropout                   0.9
layers        noise          
[200, 60, 40] 0.0    0.692095
              0.5    0.490584
              1.0    0.704022
              10.0   0.693709
              100.0  0.688343

In [18]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).max()

KeyError: 'lr'

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')